<a href="https://colab.research.google.com/github/Nadira173/DATA_AMS/blob/main/Preprocessing_Data_AMS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#IMPORT DATA

Untuk data mengenai keputusan MK terhadap sekolah negeri dan swasta gratis kami melakukan scrapping data dari X dan youtube API.

In [1]:
!pip install squarify
!pip install Sastrawi
!pip install unidecode

import itertools, re, pickle
import time, operator, numpy as np, matplotlib.pyplot as plt, pandas as pd, seaborn as sns
import networkx as nx
import squarify
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.colors as mcolors
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')

from nltk.tokenize import word_tokenize
from matplotlib.colors import ListedColormap
from textblob import TextBlob
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from unidecode import unidecode
from nltk.tag import CRFTagger
from html import unescape
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from tqdm import tqdm
from wordcloud import WordCloud

"DONE!!!"

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


'DONE!!!'

In [2]:
import pandas as pd
url = "https://raw.githubusercontent.com/Nadira173/DATA_AMS/main/data%20fix%20-%20fulldata_with_sentiment.csv"

try:
    df = pd.read_csv(url)
    print(df.head())
except Exception as e:
    print("Terjadi kesalahan saat membaca file:", e)

                                                text sentiment username  \
0  MK Putuskan SD-SMP Swasta Gratis Komisi X DPR ...   neutral      NaN   
1  Komisi X DPR RI berkomitmen mengawal implement...   neutral      NaN   
2  MK Putuskan SD-SMA Negeri dan Swasta Gratis Pa...   neutral      NaN   
3  Respons Wamendagri Bima Arya soal Pendidikan G...   neutral      NaN   
4  Respons Pemkab Bantul Soal Putusan MK tentang ...   neutral      NaN   

                       created_at  likes  reply_count  \
0  Fri May 30 04:22:59 +0000 2025    1.0            0   
1  Fri May 30 04:19:07 +0000 2025    0.0            0   
2  Fri May 30 04:11:28 +0000 2025    3.0            2   
3  Fri May 30 04:03:08 +0000 2025    3.0            0   
4  Fri May 30 03:50:46 +0000 2025    0.0            0   

                                                 url  
0  https://x.com/undefined/status/192830611885126...  
1  https://x.com/undefined/status/192830514333582...  
2  https://x.com/undefined/status/1928303

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5366 entries, 0 to 5365
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   text         5366 non-null   object 
 1   sentiment    5361 non-null   object 
 2   username     3699 non-null   object 
 3   created_at   5366 non-null   object 
 4   likes        5366 non-null   float64
 5   reply_count  5366 non-null   int64  
 6   url          1667 non-null   object 
dtypes: float64(1), int64(1), object(5)
memory usage: 293.6+ KB


In [4]:
df.columns

Index(['text', 'sentiment', 'username', 'created_at', 'likes', 'reply_count',
       'url'],
      dtype='object')

In [5]:
# Mengakses kolom 'sentiment' dari DataFrame df dan mendapatkan nilai unik
df['sentiment'].unique()

array(['neutral', 'negative', 'positive', nan, 'Positive'], dtype=object)

In [6]:
df['sentiment'] = df['sentiment'].replace('Positive', 'positive')



In [7]:
df['sentiment'].unique()

array(['neutral', 'negative', 'positive', nan], dtype=object)

In [8]:
# Mapping dari string ke angka
label_mapping = {
    'positive, Positive': 1,
    'neutral': 2,
    'negative': 3
}

df['sentiment_num'] = df['sentiment'].map(label_mapping)

print(df)

                                                   text sentiment  \
0     MK Putuskan SD-SMP Swasta Gratis Komisi X DPR ...   neutral   
1     Komisi X DPR RI berkomitmen mengawal implement...   neutral   
2     MK Putuskan SD-SMA Negeri dan Swasta Gratis Pa...   neutral   
3     Respons Wamendagri Bima Arya soal Pendidikan G...   neutral   
4     Respons Pemkab Bantul Soal Putusan MK tentang ...   neutral   
...                                                 ...       ...   
5361       Bagus itu gak semuanya org punya uang semoga  negative   
5362  Nanti ukungnya di korop kalao semua di bebanka...  negative   
5363  Hampang sj! Dana MBG alihkan sepenuhnya ke bia...  negative   
5364  jika APBN TIDAK DIKORUPSI ,  maka sekolah Nege...   neutral   
5365  Iya. Jika tidak ada korupsi.... sekolah negeri...  negative   

                    username                      created_at  likes  \
0                        NaN  Fri May 30 04:22:59 +0000 2025    1.0   
1                        NaN 

In [9]:
df["created_at"] = pd.to_datetime(df["created_at"], errors='coerce', utc=True).dt.tz_localize(None)

<ipython-input-9-5d4955c7d875>:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["created_at"] = pd.to_datetime(df["created_at"], errors='coerce', utc=True).dt.tz_localize(None)


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5366 entries, 0 to 5365
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   text           5366 non-null   object        
 1   sentiment      5361 non-null   object        
 2   username       3699 non-null   object        
 3   created_at     5366 non-null   datetime64[ns]
 4   likes          5366 non-null   float64       
 5   reply_count    5366 non-null   int64         
 6   url            1667 non-null   object        
 7   sentiment_num  4507 non-null   float64       
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 335.5+ KB


#Slang

In [11]:
import json
import json

# Membaca file slang.json dari /content/
with open('/content/slang_cleaned_recovered.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# Menampilkan data
data
# Menampilkan data
data


{'bakalan': 'akan',
 'ngincer': 'mengincar',
 'ga': 'tidak',
 'tolol': 'bodoh',
 'sampe': 'sampai',
 'dll': 'dan lain-lain',
 'duit': 'uang',
 'ma': 'Madrasah Aliyah',
 'herman': 'heran',
 'utk': 'untuk',
 'tdk': 'tidak',
 'tsb': 'tersebut',
 'yg': 'yang',
 'ngomong': 'berbicara',
 'g': 'tidak',
 'udah': 'sudah',
 'aja': 'saja',
 'disitu': 'disana',
 'jg': 'juga',
 'ortu': 'orang tua',
 'milih': 'memilih',
 'nggak': 'tidak',
 'pem': 'pemerintah',
 'ngerasain': 'merasakan',
 'mau': 'ingin',
 'doang': 'saja',
 'jd': 'jadi',
 'emang': 'memang',
 'loe': 'kamu',
 'pembamgunan': 'pembangunan',
 'gak': 'tidak',
 'gw': 'saya',
 'bikin': 'membuat',
 'jgn': 'jangan',
 'kls': 'kelas',
 'dapet': 'dapat',
 'ky': 'kayak',
 'klo': 'kalau',
 'dgratisin': 'digratiskan',
 'ngemeng': 'ngomong',
 'nnti': 'nanti',
 'gimana': 'bagaimana',
 'bilangnya': 'mengatakannya',
 'dr': 'dari',
 'ngga': 'tidak',
 'skrg': 'sekarang',
 'dikasih': 'diberikan',
 'situ': 'kamu',
 'gratisin': 'gratiskan',
 'gk': 'tidak',
 '

In [12]:
import re
# Melakukan penanganan slang/singkatan
def replace_slang(text,dict_slang):
    # Mengganti frasa menggunakan regex
    for slang, pengganti in dict_slang.items():
        text = re.sub(r'\b' + re.escape(slang) + r'\b', pengganti, text) # versi tanpa ignorecase, sensitif dengan huruf kapital atau kecil
    return text

In [13]:
df['new'] = df['text'].apply(lambda x: replace_slang(x, data))

In [15]:
df

,text,sentiment,username,created_at,likes,reply_count,url,sentiment_num,new
0,MK Putuskan SD-SMP Swasta Gratis Komisi X DPR ...,neutral,NaN,2025-05-30 04:22:59,1.0,0,https://x.com/undefined/status/192830611885126...,2.0,MK Putuskan Sekolah Dasar-Sekolah Menengah Per...
1,Komisi X DPR RI berkomitmen mengawal implement...,neutral,NaN,2025-05-30 04:19:07,0.0,0,https://x.com/undefined/status/192830514333582...,2.0,Komisi X DPR RI berkomitmen mengawal implement...
2,MK Putuskan SD-SMA Negeri dan Swasta Gratis Pa...,neutral,NaN,2025-05-30 04:11:28,3.0,2,https://x.com/undefined/status/192830322054963...,2.0,MK Putuskan Sekolah Dasar-Sekolah Menengah Ata...
3,Respons Wamendagri Bima Arya soal Pendidikan G...,neutral,NaN,2025-05-30 04:03:08,3.0,0,https://x.com/undefined/status/192830112260070...,2.0,Respons Wamendagri Bima Arya soal Pendidikan G...
4,Respons Pemkab Bantul Soal Putusan MK tentang ...,neutral,NaN,2025-05-30 03:50:46,0.0,0,https://x.com/undefined/status/192829801043108...,2.0,Respons Pemkab Bantul Soal Putusan MK tentang ...
...,...,...,...,...,...,...,...,...,...
5361,Bagus itu gak semuanya org punya uang semoga,negative,@ArdhiROBLOXCHANNEL,2025-05-30 02:37:07,0.0,0,NaN,3.0,Bagus itu tidak semuanya orang punya uang semoga
5362,Nanti ukungnya di korop kalao semua di bebanka...,negative,@hambali8442,2025-05-30 02:36:47,0.0,0,NaN,3.0,Nanti ukungnya di korop kalao semua di bebanka...
5363,Hampang sj! Dana MBG alihkan sepenuhnya ke bia...,negative,@muhamadsmithwayne6496,2025-05-30 02:32:14,0.0,0,NaN,3.0,Hampang saja! Dana Makan Bergizi Gratis alihka...
5364,"jika APBN TIDAK DIKORUPSI , maka sekolah Nege...",neutral,@TimPublikasiFHO,2025-05-30 02:30:08,1.0,1,NaN,2.0,"jika APBN TIDAK DIKORUPSI , maka sekolah Nege..."


#Normalisasi

In [16]:
def remove_URL(tweet): # Menghapus URL
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'', tweet)

def remove_html(tweet): # Menghapus tag HTML
    html = re.compile(r'<.*?>')
    return html.sub(r'', tweet)

def remove_emoji(tweet): # Menghapus emoji
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"
        u"\U0001F300-\U0001F5FF"
        u"\U0001F680-\U0001F6FF"
        u"\U0001F1E0-\U0001F1FF"
                            "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', tweet)

def remove_username(tweet): # Menghapus username
    username_pattern = re.compile(r'@\w+')
    return username_pattern.sub(r'', tweet)

def remove_numbers (tweet): # Menghapus angka
    tweet = re.sub(r'\d+', '', tweet)
    return tweet

def remove_symbols (tweet): # Menghapus simbol
    tweet = re.sub(r'[^a-zA-Z0-9\s]', '', tweet)
    return tweet

def case_folding(text): # Mengubah huruf yang masih uppercase menjadi lowercase
  if isinstance(text, str):
    lowercase_text = text.lower()
    return lowercase_text
  else:
    return text

In [17]:
# Visualisasi clean dataset 'full_text'
df['cleaning'] = df['new'].apply(lambda x: remove_URL(x))
df['cleaning'] = df['cleaning'].apply(lambda x: remove_html(x))
df['cleaning'] = df['cleaning'].apply(lambda x: remove_emoji(x))
df['cleaning'] = df['cleaning'].apply(lambda x: remove_username(x))
df['cleaning'] = df['cleaning'].apply(lambda x: remove_symbols(x))
df['cleaning'] = df['cleaning'].apply(lambda x: remove_numbers(x))
df['cleaning'] = df['cleaning'].apply(lambda x: case_folding(x))

df

,text,sentiment,username,created_at,likes,reply_count,url,sentiment_num,new,cleaning
0,MK Putuskan SD-SMP Swasta Gratis Komisi X DPR ...,neutral,NaN,2025-05-30 04:22:59,1.0,0,https://x.com/undefined/status/192830611885126...,2.0,MK Putuskan Sekolah Dasar-Sekolah Menengah Per...,mk putuskan sekolah dasarsekolah menengah pert...
1,Komisi X DPR RI berkomitmen mengawal implement...,neutral,NaN,2025-05-30 04:19:07,0.0,0,https://x.com/undefined/status/192830514333582...,2.0,Komisi X DPR RI berkomitmen mengawal implement...,komisi x dpr ri berkomitmen mengawal implement...
2,MK Putuskan SD-SMA Negeri dan Swasta Gratis Pa...,neutral,NaN,2025-05-30 04:11:28,3.0,2,https://x.com/undefined/status/192830322054963...,2.0,MK Putuskan Sekolah Dasar-Sekolah Menengah Ata...,mk putuskan sekolah dasarsekolah menengah atas...
3,Respons Wamendagri Bima Arya soal Pendidikan G...,neutral,NaN,2025-05-30 04:03:08,3.0,0,https://x.com/undefined/status/192830112260070...,2.0,Respons Wamendagri Bima Arya soal Pendidikan G...,respons wamendagri bima arya soal pendidikan g...
4,Respons Pemkab Bantul Soal Putusan MK tentang ...,neutral,NaN,2025-05-30 03:50:46,0.0,0,https://x.com/undefined/status/192829801043108...,2.0,Respons Pemkab Bantul Soal Putusan MK tentang ...,respons pemkab bantul soal putusan mk tentang ...
...,...,...,...,...,...,...,...,...,...,...
5361,Bagus itu gak semuanya org punya uang semoga,negative,@ArdhiROBLOXCHANNEL,2025-05-30 02:37:07,0.0,0,NaN,3.0,Bagus itu tidak semuanya orang punya uang semoga,bagus itu tidak semuanya orang punya uang semoga
5362,Nanti ukungnya di korop kalao semua di bebanka...,negative,@hambali8442,2025-05-30 02:36:47,0.0,0,NaN,3.0,Nanti ukungnya di korop kalao semua di bebanka...,nanti ukungnya di korop kalao semua di bebanka...
5363,Hampang sj! Dana MBG alihkan sepenuhnya ke bia...,negative,@muhamadsmithwayne6496,2025-05-30 02:32:14,0.0,0,NaN,3.0,Hampang saja! Dana Makan Bergizi Gratis alihka...,hampang saja dana makan bergizi gratis alihkan...
5364,"jika APBN TIDAK DIKORUPSI , maka sekolah Nege...",neutral,@TimPublikasiFHO,2025-05-30 02:30:08,1.0,1,NaN,2.0,"jika APBN TIDAK DIKORUPSI , maka sekolah Nege...",jika apbn tidak dikorupsi maka sekolah neger...


#Stopword

In [18]:
!wget -P data/ https://raw.githubusercontent.com/taudataanalytics/eLearning/refs/heads/master/data/stopwords_id.txt
!wget -P data/ https://raw.githubusercontent.com/louisowen6/NLP_bahasa_resources/master/combined_stop_words.txt

--2025-06-05 08:12:56--  https://raw.githubusercontent.com/taudataanalytics/eLearning/refs/heads/master/data/stopwords_id.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6446 (6.3K) [text/plain]
Saving to: ‘data/stopwords_id.txt.4’

stopwords_id.txt.4  100%[===================>]   6.29K  --.-KB/s    in 0s      

2025-06-05 08:12:56 (29.2 MB/s) - ‘data/stopwords_id.txt.4’ saved [6446/6446]

--2025-06-05 08:12:56--  https://raw.githubusercontent.com/louisowen6/NLP_bahasa_resources/master/combined_stop_words.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, await

In [19]:
# Membaca file stopwords
stopword_files = ['data/stopwords_id.txt', 'data/combined_stop_words.txt']
id_stop = []
for stopword_file in stopword_files:
    with open(stopword_file, "r", encoding="utf-8", errors='replace') as file:
        id_stop.extend([word.strip().lower() for word in file.readlines()])


# Bersihkan stopwords (hapus spasi dan ubah ke huruf kecil)
id_stop = [word for word in id_stop if word] # remove empty strings

In [20]:
# Tambahkan stopwords tambahan
additional_stopwords = {'gk','nggak''aja', 'dong', 'deh', 'nih', 'sih', 'kok', 'kayak', 'makanya', 'nggak',
    'gak', 'ga', 'gue', 'gua', 'lu', 'loe', 'lho', 'ya', 'si', 'dong', 'doang',
    'malah', 'banget', 'nya', 'nyaa', 'nih', 'tuh', 'lah', 'kan', 'pun', 'mah',
    'cuma', 'padahal', 'seharusnya', 'soalnya', 'biar', 'biarpun', 'bakal',
    'bakalan', 'semoga', 'moga', 'mudah2an', 'ntar', 'nanti', 'sama', 'dengan',
    'tanpa', 'dulu', 'lagi', 'terus', 'udah', 'sudah', 'belum', 'baru', 'mau',
    'akan', 'adalah', 'ialah', 'itu', 'ini', 'tersebut', 'tsb', 'anda', 'kamu',
    'saya', 'aku', 'mereka', 'kita', 'kami', 'engkau', 'siapa', 'apa', 'dimana',
    'kenapa', 'mengapa', 'bagaimana', 'yang', 'di', 'ke', 'dari', 'pada', 'buat',
    'untuk', 'agar', 'karena', 'sebab', 'walaupun', 'meskipun', 'hingga', 'sejak',
    'sebelum', 'sesudah', 'tetapi', 'namun', 'dan', 'atau', 'juga', 'kalau',
    'jika', 'bila', 'hingga', 'supaya', 'sehingga', 'oleh', 'tentang', 'terhadap',
    'dalam', 'atas', 'bagi', 'antara', 'selama', 'serta', 'bahwa'}
id_stop.extend(additional_stopwords)  # Gabungkan stopwords tambahan

# Fungsi untuk menghapus stopwords
def remove_custom_stopwords(text):
    words = word_tokenize(text)  # Tokenisasi teks
    return ' '.join([word for word in words if word not in id_stop])

In [21]:
df['no_stopwords'] = df['cleaning'].apply(remove_custom_stopwords)
df

,text,sentiment,username,created_at,likes,reply_count,url,sentiment_num,new,cleaning,no_stopwords
0,MK Putuskan SD-SMP Swasta Gratis Komisi X DPR ...,neutral,NaN,2025-05-30 04:22:59,1.0,0,https://x.com/undefined/status/192830611885126...,2.0,MK Putuskan Sekolah Dasar-Sekolah Menengah Per...,mk putuskan sekolah dasarsekolah menengah pert...,mk putuskan sekolah dasarsekolah menengah swas...
1,Komisi X DPR RI berkomitmen mengawal implement...,neutral,NaN,2025-05-30 04:19:07,0.0,0,https://x.com/undefined/status/192830514333582...,2.0,Komisi X DPR RI berkomitmen mengawal implement...,komisi x dpr ri berkomitmen mengawal implement...,komisi x dpr ri berkomitmen mengawal implement...
2,MK Putuskan SD-SMA Negeri dan Swasta Gratis Pa...,neutral,NaN,2025-05-30 04:11:28,3.0,2,https://x.com/undefined/status/192830322054963...,2.0,MK Putuskan Sekolah Dasar-Sekolah Menengah Ata...,mk putuskan sekolah dasarsekolah menengah atas...,mk putuskan sekolah dasarsekolah menengah nege...
3,Respons Wamendagri Bima Arya soal Pendidikan G...,neutral,NaN,2025-05-30 04:03:08,3.0,0,https://x.com/undefined/status/192830112260070...,2.0,Respons Wamendagri Bima Arya soal Pendidikan G...,respons wamendagri bima arya soal pendidikan g...,respons wamendagri bima arya pendidikan gratis...
4,Respons Pemkab Bantul Soal Putusan MK tentang ...,neutral,NaN,2025-05-30 03:50:46,0.0,0,https://x.com/undefined/status/192829801043108...,2.0,Respons Pemkab Bantul Soal Putusan MK tentang ...,respons pemkab bantul soal putusan mk tentang ...,respons pemkab bantul putusan mk sekolah grati...
...,...,...,...,...,...,...,...,...,...,...,...
5361,Bagus itu gak semuanya org punya uang semoga,negative,@ArdhiROBLOXCHANNEL,2025-05-30 02:37:07,0.0,0,NaN,3.0,Bagus itu tidak semuanya orang punya uang semoga,bagus itu tidak semuanya orang punya uang semoga,bagus uang
5362,Nanti ukungnya di korop kalao semua di bebanka...,negative,@hambali8442,2025-05-30 02:36:47,0.0,0,NaN,3.0,Nanti ukungnya di korop kalao semua di bebanka...,nanti ukungnya di korop kalao semua di bebanka...,ukungnya korop kalao bebankan pemerintah danan...
5363,Hampang sj! Dana MBG alihkan sepenuhnya ke bia...,negative,@muhamadsmithwayne6496,2025-05-30 02:32:14,0.0,0,NaN,3.0,Hampang saja! Dana Makan Bergizi Gratis alihka...,hampang saja dana makan bergizi gratis alihkan...,hampang dana makan bergizi gratis alihkan sepe...
5364,"jika APBN TIDAK DIKORUPSI , maka sekolah Nege...",neutral,@TimPublikasiFHO,2025-05-30 02:30:08,1.0,1,NaN,2.0,"jika APBN TIDAK DIKORUPSI , maka sekolah Nege...",jika apbn tidak dikorupsi maka sekolah neger...,apbn dikorupsi sekolah negeri swasta gratis


#Lemmatization

In [22]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()

In [23]:
kata_dasar = [
    'setuju', 'sukses', 'terbaik', 'pemerataan', 'lingkungan', 'setara', 'alasan',
    'pendidikan', 'peserta', 'impian', 'kejuruan', 'menteri', 'terapkan', 'perdebatan',
    'hilangkan', 'biarkan','kualitas', 'akses', 'program', 'pengembangan', 'pemerintah', 'dukungan', 'kebijakan',
    'pelayanan', 'pemberdayaan', 'pelajar', 'guru', 'inovasi', 'fasilitas', 'pembelajaran',
    'kebutuhan', 'pelatihan', 'partisipasi', 'evaluasi', 'pelaksanaan', 'keadilan',
    'motivasi', 'hasil', 'pendampingan', 'pengawasan', 'pengelolaan', 'standar',
    'pengaruh', 'peran', 'kesempatan', 'tantangan', 'penguatan', 'kolaborasi', 'solusi',
    'transparansi', 'akreditasi', 'komitmen', 'perbaikan', 'kepemimpinan'
]


In [24]:
# Fungsi stemming dengan perlindungan kata dasar
def stem_text(text):
    words = text.split()
    stemmed_words = []
    for word in words:
        stemmed = stemmer.stem(word)
        # Jika kata asli ada di kata_dasar, gunakan kata asli
        if word in kata_dasar:
            stemmed_words.append(word)
        # Jika hasil stemming malah menghapus bentuk dasar yang kamu inginkan
        elif stemmed in kata_dasar:
            stemmed_words.append(stemmed)
        else:
            stemmed_words.append(stemmed)
    return ' '.join(stemmed_words)

In [25]:
df['stemmed_text'] = df['no_stopwords'].apply(stem_text)

In [26]:
df

,text,sentiment,username,created_at,likes,reply_count,url,sentiment_num,new,cleaning,no_stopwords,stemmed_text
0,MK Putuskan SD-SMP Swasta Gratis Komisi X DPR ...,neutral,NaN,2025-05-30 04:22:59,1.0,0,https://x.com/undefined/status/192830611885126...,2.0,MK Putuskan Sekolah Dasar-Sekolah Menengah Per...,mk putuskan sekolah dasarsekolah menengah pert...,mk putuskan sekolah dasarsekolah menengah swas...,mk putus sekolah dasarsekolah tengah swasta gr...
1,Komisi X DPR RI berkomitmen mengawal implement...,neutral,NaN,2025-05-30 04:19:07,0.0,0,https://x.com/undefined/status/192830514333582...,2.0,Komisi X DPR RI berkomitmen mengawal implement...,komisi x dpr ri berkomitmen mengawal implement...,komisi x dpr ri berkomitmen mengawal implement...,komisi x dpr ri komitmen awal implementasi put...
2,MK Putuskan SD-SMA Negeri dan Swasta Gratis Pa...,neutral,NaN,2025-05-30 04:11:28,3.0,2,https://x.com/undefined/status/192830322054963...,2.0,MK Putuskan Sekolah Dasar-Sekolah Menengah Ata...,mk putuskan sekolah dasarsekolah menengah atas...,mk putuskan sekolah dasarsekolah menengah nege...,mk putus sekolah dasarsekolah tengah negeri sw...
3,Respons Wamendagri Bima Arya soal Pendidikan G...,neutral,NaN,2025-05-30 04:03:08,3.0,0,https://x.com/undefined/status/192830112260070...,2.0,Respons Wamendagri Bima Arya soal Pendidikan G...,respons wamendagri bima arya soal pendidikan g...,respons wamendagri bima arya pendidikan gratis...,respons wamendagri bima arya pendidikan gratis...
4,Respons Pemkab Bantul Soal Putusan MK tentang ...,neutral,NaN,2025-05-30 03:50:46,0.0,0,https://x.com/undefined/status/192829801043108...,2.0,Respons Pemkab Bantul Soal Putusan MK tentang ...,respons pemkab bantul soal putusan mk tentang ...,respons pemkab bantul putusan mk sekolah grati...,respons pemkab bantul putus mk sekolah gratis ...
...,...,...,...,...,...,...,...,...,...,...,...,...
5361,Bagus itu gak semuanya org punya uang semoga,negative,@ArdhiROBLOXCHANNEL,2025-05-30 02:37:07,0.0,0,NaN,3.0,Bagus itu tidak semuanya orang punya uang semoga,bagus itu tidak semuanya orang punya uang semoga,bagus uang,bagus uang
5362,Nanti ukungnya di korop kalao semua di bebanka...,negative,@hambali8442,2025-05-30 02:36:47,0.0,0,NaN,3.0,Nanti ukungnya di korop kalao semua di bebanka...,nanti ukungnya di korop kalao semua di bebanka...,ukungnya korop kalao bebankan pemerintah danan...,ukungnya korop kalao beban pemerintah dana atu...
5363,Hampang sj! Dana MBG alihkan sepenuhnya ke bia...,negative,@muhamadsmithwayne6496,2025-05-30 02:32:14,0.0,0,NaN,3.0,Hampang saja! Dana Makan Bergizi Gratis alihka...,hampang saja dana makan bergizi gratis alihkan...,hampang dana makan bergizi gratis alihkan sepe...,hampang dana makan gizi gratis alih sepenuh bi...
5364,"jika APBN TIDAK DIKORUPSI , maka sekolah Nege...",neutral,@TimPublikasiFHO,2025-05-30 02:30:08,1.0,1,NaN,2.0,"jika APBN TIDAK DIKORUPSI , maka sekolah Nege...",jika apbn tidak dikorupsi maka sekolah neger...,apbn dikorupsi sekolah negeri swasta gratis,apbn korupsi sekolah negeri swasta gratis


In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5366 entries, 0 to 5365
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   text           5366 non-null   object        
 1   sentiment      5361 non-null   object        
 2   username       3699 non-null   object        
 3   created_at     5366 non-null   datetime64[ns]
 4   likes          5366 non-null   float64       
 5   reply_count    5366 non-null   int64         
 6   url            1667 non-null   object        
 7   sentiment_num  4507 non-null   float64       
 8   new            5366 non-null   object        
 9   cleaning       5366 non-null   object        
 10  no_stopwords   5366 non-null   object        
 11  stemmed_text   5366 non-null   object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(8)
memory usage: 503.2+ KB


In [30]:
df.to_csv("hasil prepro data fix.csv", index=False)


#Visualisasi